# ISSN matching

This notebook contains a probabilistic matching between metadata of periodicals from the Royal Library of Belgium (KBR) and a dataset of Belgian periodicals from the ISSN center. The aim is to enrich KBR records with their corresponding ISSN number. The matching is performed via Splink https://moj-analytical-services.github.io/splink.

In the following we follow the Splink tutorial, beginning with extracting relevant CSV fields from the input XML data and standardizing it.

In [1]:
# setting to autoreload Python files if they have changed
%load_ext autoreload
%autoreload 2
    
import utils
import pandas as pd


In [2]:
from splink.exploratory import completeness_chart
from splink import DuckDBAPI
db_api = DuckDBAPI()

In [69]:
commonColumns = ['title', 'alternateTitle', 'publisher', 'place', 'place-enriched', 'udc', 'url']
trainedModelFilename = 'trained-splink-model.json'

## 1 Get data

In the first step we have to retrieve the data. Both data sources provide data in the strcutured XML format from which we will extract relevant fields into CSV files, the expected input format for Splink.

### 1.1 Get KBR data
With the back-end catalog system Syracuse we can export MARCXML metadata about all Belgian paper periodicals that do not already have an ISSN with the search query: `TYPN=PERE AND ISSN="" AND ALIE="Belg*"`.

The records have many relevant fields that could lead to potential matches
* `035$a`: OCoLC identifier
* `041$a`: language of document
* `100$a`: author name (`100$*` for linked author authority)
* `245$a`: title
* `245$b`: remainder of title
* `245$c`: responsibility statement
* `264$a`: place of publication
* `264$b`: name of publisher
* `264$c`: date of production
* `490$a`: series statement
* `650$a`: subject index term (`$*` for linked subject index authority, Belgian Bibliography or FAST)
* `653$a`: Index Term-Uncontrolled ????
* `710$a`: Linked organization authorities (e.g. publishers and printers, separately indicated via MARC relator code in `$4`)
* `856$u`: URL

The configuration for the extraction script is in the following JSON file: [extract-config-kbr.json](./extract-config-kbr.json)
and it is used in the bash script `extract-kbr.sh` that contains the parameterized call to xml-to-csv.

In [4]:
# bash extract-kbr.sh

The xml-to-csv script creates a main CSV file and because one data field may occur more than once, the script also creates a separate csv file per data field to resolve 1:n relationships. For example, if record `A1` has the following two alternate titles `the English title` and `de Nederlandse titel`, then the main CSV will contain a single row with id `A1` and the following in the column alternateTitel `["the English title", "de Nederlandse titel"]`, whereas the separate CSV file for alternateTitel contains two rows with id `A1` and each row with a single title only.
We load both the mainCSV as well as all the column CSV files into dataframes (dfListKBRColumns is a dictionary of dataframes).

In [5]:
dfKBR, dfListKBRColumns = utils.createInputDataframe('kbr-data', 'kbr', commonColumns + ['titleRemainder'])

In [6]:
# number of rows
dfKBR.shape[0]

26822

### 1.2 Get ISSN-plus data
As ISSN national center, we could create an export of Belgian periodicals via a web interface.

This data is rather limited, the following relevant fields exist:
* `080$a`: Universal Decimal Classification
* `210$a`: Short title
* `245$a`: title
* `260$a`: place of publication
* `260$b`: name of publisher
* `856$u`: URL

The configuration for the extraction script is in the following JSON file: [extract-config-issn-plus.json](./extract-config-issn-plus.json)
and it is used in the bash script `extract-issn-plus.sh` that contains the parameterized call to xml-to-csv.

In [7]:
# bash extract-issn-plus.sh

see previous section for details about the xml-to-csv extraction and what the following dataframes contain

In [8]:
dfISSN, dfListISSNColumns = utils.createInputDataframe('issn-plus-data', 'issn-plus', commonColumns + ['keyTitle'])

In [9]:
# number of rows
dfISSN.shape[0]

36565

### 1.3 Overlap and strategy

Both datasets need to have the same columns for Splink, additionally the data should be uniformized as much as possible.
In this section we list which fields we extract and what our overal strategy will be, in the next section we focus on the standardization/uniformization of the data.

We use the following columns

* unique_identifier (unique row number, see explanation below)
* identifier (the KBR IDN for KBR, and the ISSN number foor ISSN+ records)
* title (a title variant)
* place of publication
* publisher
* classification
* url
* data source

In the following section we describe and apply the necessary pre-processing for standardization and uniformization.

## 2 Standardization and Uniformization

In this section we elaborate how we uniformize the columns from the two data sources.
This is a necessary prerequisite of the Splink data linkage.
In addition to the mentioned columns, we will have the column `identifier` that in case of KBR will be the KBR IDN and in case of ISSN+ will be the ISSN number). Additionally we will create the column `unique_identifier` after the pre-processing, this one is used to uniquely identify each row even for cases where we have to extend a certain record from KBR or ISSN+ into several rows (e.g. because there is more than one title variant).

### 2.1 Title

Both, KBR and ISSN+ provide different kind of title fields. There is no uniform way to have all of them meaningfully in separate columns. For example, at KBR the "main title" might be in Dutch and one of the alternative titles in French, whereas for ISSN+ it could be the other way around. Hence we will use a single title column and hence create multiple rows per periodical. Each row will get a unique row number.
To reduce the number of comparisons and limit the comparisons to meaningful ones, we will configure Splink to only compare rows with a different value in the `data source` column.
Thus for the periodical `Aspects techniques de la sécurité routière : fiches documentaires` we will get four rows with the titles

* `Aspects techniques de la sécurité routière : fiches documentaires`
* `Technical aspects of road safety : documentation cards`
* `Technische aspecten der verkeersveiligheid : documentatie steekkaarten`
* `Technische Seiten der Verkehrssichreheit : Dokumentations Karteikarten`

Please note that in this case the German spelling contains several smaller mistakes, but this will be taken care of during the fuzzy matching with Flink.

All of those titles will be matched with all of the ISSN+ titles. However, we keep the KBR identifier, hence we later on can select the variant with the highest matching probability to basically reduce the number of results again.


In [10]:
# title normalization KBR

In [11]:
dfKBRMatching = dfKBR[['autID']].copy()

In [12]:
dfKBRMatching['dataSource'] = 'KBR'

In [13]:
dfKBRMatching

,autID,dataSource
0,15145431,KBR
1,18584980,KBR
2,18585150,KBR
3,16474587,KBR
4,16474568,KBR
...,...,...
26817,18443022,KBR
26818,22482029,KBR
26819,15288181,KBR
26820,16796898,KBR


In [14]:
kbrAllTitles = pd.concat([
    dfListKBRColumns['title'][['autID', 'title']],
    dfListKBRColumns['alternateTitle'][['autID', 'alternateTitle']].rename(columns={'alternateTitle': 'title'}),
    dfListKBRColumns['titleRemainder'][['autID', 'titleRemainder']].rename(columns={'titleRemainder': 'title'}),
], ignore_index=True)


In [15]:
kbrAllTitles

,autID,title
0,15145431,Bulletin de la Société belge d'études géograph...
1,18584980,Voorname aanwinsten
2,18585150,Voorname aanwinsten
3,16474587,Voorname aanwinsten
4,16474568,Acquisitions majeures
...,...,...
38817,15869154,revue mensuelle publiée par la province de Hai...
38818,18570211,revue trimestriëlle
38819,15952800,revue d'art et de littérature
38820,18443022,1908-1983


In [16]:
dfKBRMatching = dfKBRMatching.merge(kbrAllTitles, how='left', left_on='autID', right_on='autID')

In [17]:
# number of all possible titles
dfKBRMatching.shape[0]

38824

For ISSN+ we also want to use the `keyTitle` field.

In [18]:
# title normalization ISSN+
dfISSNMatching = dfISSN[['autID']].copy()
dfISSNMatching['dataSource'] = 'ISSN'

In [19]:
issnAllTitles = pd.concat([
    dfListISSNColumns['title'][['autID', 'title']],
    dfListISSNColumns['alternateTitle'][['autID', 'alternateTitle']].rename(columns={'alternateTitle': 'title'}),
    dfListISSNColumns['keyTitle'][['autID', 'keyTitle']].rename(columns={'keyTitle': 'title'}),
], ignore_index=True)

In [20]:
dfISSNMatching = dfISSNMatching.merge(issnAllTitles, how='left', left_on='autID', right_on='autID')

In [21]:
# number of all possible titles
dfISSNMatching.shape[0]

84937

In [22]:
dfMatching = pd.concat([dfKBRMatching,dfISSNMatching])

Last but not least we will normalize the final column (lower case, special characters etc)

In [23]:
dfMatching = utils.normalizeColumn(dfMatching, 'title')

In [24]:
dfMatching

,autID,dataSource,title
0,15145431,KBR,bulletin de la societe belge d'etudes geograph...
1,15145431,KBR,tijdschrift van de belgische vereniging voor a...
2,15145431,KBR,bevas
3,15145431,KBR,sobeg
4,18584980,KBR,voorname aanwinsten
...,...,...,...
84932,3041-7619,ISSN,j.v.k.a. orgaan voor katholieke studentenaktie
84933,3041-7619,ISSN,lenteweelde. j.v.k.a. orgaan voor katholieke s...
84934,3041-8038,ISSN,jeugdland.
84935,3041-8038,ISSN,voor klein en groot


### 2.2 Place of publication

The column place in the KBR and ISSN dataframe contains place names in different languages, e.g. `Anvers` (FR) or `Antwerpen` (NL) which both refer to the city of `Antwerp`(EN). We use a local GeoNames database and API to retrieve the uniform English name.

We ran the bash script `enrich-geonames-kbr.sh` that utilizes the script [geoname-enrichment](https://github.com/MetaBelgica/geoname-enrichment) (which makes use of an [internal API](https://github.com/kbrbe/geonames-lookup)). From `24,673` KBR records with place name , `20,002` could be enriched in `16` minutes. for `3,497` no GeoNames ID was found and for `1,173` multiple API results were reported.
**Update** we have encountered that some records mentioned several place names in one field, delimited by semicolon. After an update of the xml-to-csv software (https://github.com/kbrbe/xml-to-csv/releases/tag/v0.1.5) we could extract `24,727` from which `20,064` could be enriched. 

The places were nothing was found for, often are foreign places for which we did not have country information, such as Paris or Montréal (for all records in the dump the country Belgium was assumed, hence no GeoNames ID was found for `Paris (Belgium)`).
Additionally, common unclean data include are two languages in one field, e.g. `Bruxelles = Brussel`, or `Brussel = Bruxelles`, but also values like `= Bruxelles`.


In [25]:
# read GeoName enriched KBR data
dfNormKBRPlaces = pd.read_csv('kbr/kbr-data-place-enriched.csv')

Similarly we enriched placenames for ISSN+. From the `38,589` records that passed the physical periodicals filter, `37,462` contained a place name (`89%`). The script enriched `33,346` records in `27` minutes (`22` records per second). For `1,424` no GeoNames ID was found and for `2,691` multiple API results were reported. 
**Update** we have encountered that some records mentioned several place names in one field, delimited by semicolon. After an update of the xml-to-csv software (https://github.com/kbrbe/xml-to-csv/releases/tag/v0.1.5) we could extract `37,494` from which `33,375` could be enriched. 

Similar as above, common issues were foreign place names. Also places were nothing was found for, sometimes are recently merged municipalities such as
* Nazareth-De Pinte
* Pajottegem
* Tongeren-Borgloon


This step had the aim to uniformize the spelling and not to enrich with a GeoNames identifier. This means that actually more than the `89%` of records have uniformized data. For example, even though _Pajottegem_ was not found in GeoNames, this place is likely spelled the same in the whole file as there is no French translation like for the city of Antwerp. 

Please note that we cannot simply take the resulting CSV of the enriching as it only contains the rows for which a GeoNames record was found. We additionally have to merge the output list with the records for which nothing was found.



In [26]:
dfNormISSNPlaces = pd.read_csv('issn-plus/issn-plus-data-place-enriched.csv')

In [27]:
dfNormKBRPlaces

,autID,geonameID,placename,countryCode
0,15145431,2792482,Leuven,BE
1,18584980,2800866,Brussels,BE
2,18585150,2800866,Brussels,BE
3,16474587,2800866,Brussels,BE
4,16474568,2800866,Brussels,BE
...,...,...,...,...
20060,18443022,2803138,Antwerp,BE
20061,22482029,2800866,Brussels,BE
20062,15288181,2785389,Tongeren,BE
20063,16796898,2800866,Brussels,BE


In [28]:
dfPlaces = pd.concat([
    dfListKBRColumns['place'][['autID', 'place']],
    dfNormKBRPlaces[['autID', 'placename']].rename(columns={'placename': 'place'}),
    dfListISSNColumns['place'][['autID', 'place']],
    dfNormISSNPlaces[['autID', 'placename']].rename(columns={'placename': 'place'}),
], ignore_index=True)

In [29]:
dfListISSNColumns['place'][dfListISSNColumns['place']['place'].str.contains(';')]

,autID,place,country


In [30]:
dfPlaces[dfPlaces['place'].str.contains('=')]

,autID,place
44,15433824,= Brussel
55,16916241,Bruxelles = Brussel
98,17844676,= Brussel
235,18459106,Brussel = Bruxelles
238,18451213,= Brussel
...,...,...
72056,2593-8762,Brussels = Bruxelles = Brussel
76168,2983-4511,Bruxelles=Brussel
77475,2684-1452,Namen = Namur
77498,2684-1746,Bruxelles = Brussel


In [31]:
dfPlaces['place'] = dfPlaces['place'].str.lstrip('=')

In [32]:
dfPlaces['place'] = dfPlaces['place'].str.strip('[]')

In [33]:
mapping = {
    "Namur": ["Namen", "Namur"],
    "Antwerp": ["Antwerpen", "Antwerp", "Anvers"],
    "Brussels": ["Brussels", "Bruxelles", "Brüssel", "Brussel"],
    "Bruges": ["Brugge", "Bruges", "Brügge"],
    "Leuven": ["Leuven", "Louvain"],
    "Ghent": ["Gent", "Ghent", "Gand"],
    "Namur": ["Namur", "Namen"],
    "Saint-Gillis": ["Sint-Gillis", "Saint-Gillis"],
    "Liège": ["Luik", "Liège", "Lüttich"],
    "Sint-Pieters-Woluwe": ["Sint-Pieters-Woluwe"],
    "Woluwé-Saint-Lambert": ["Sint-Lambrechts-Woluwe", "Woluwé-Saint-Lambert"],
    "Kortrijk": ["Courtrai"],
    "Tienen": ["Tirlemont", "Tienen"],
    "Zottegem": ["Zottegem"],
    "Dendermonde": ["Dendermonde"]
}

In [34]:
for target, patterns in mapping.items():
    regex = "|".join(patterns)
    dfPlaces.loc[dfPlaces['place'].str.contains(regex, case=False, na=False), 'place'] = target


In [35]:
dfPlaces[dfPlaces['place'].str.contains('=')]

,autID,place
8931,17789802,Bureau des études et l'imprimerie de l'Ecole d...
10849,16913736,(1ère année = jaarg.)-
11009,16925793,9e année = 9e jaargang) - 1963 (15e année = 15...
14369,16894372,BSD = SPB
24030,15003055,Maastricht = Hasselt
24443,16458098,ensuite = daarna : Berchem


In [36]:
dfPlaces = utils.normalizeColumn(dfPlaces, "place")

In [37]:
dfPlaces

,autID,place
0,15145431,leuven
1,18584980,brussels
2,18585150,brussels
3,16474587,brussels
4,16474568,brussels
...,...,...
115656,1370-8732,liege
115657,2795-9066,brussels
115658,0779-3235,namur
115659,3041-7929,brussels


In [38]:
dfMatching = dfMatching.merge(dfPlaces,  how='left', left_on='autID', right_on='autID')

### 2.3 Publisher



In [39]:
dfListKBRColumns['publisher']

,autID,publisher
0,15145431,Universiteit van Leuven. Aardrijkskundig Insti...
1,18584980,Koninklijke Bibliotheek van België
2,18585150,Koninklijke Bibliotheek van België
3,16474587,Koninklijke Bibliotheek van België
4,16474568,Bibliothèque royale de Belgique
...,...,...
22908,22482029,Imprimerie Raymond Fischlin
22909,22482029,Imprimerie Lielens
22910,15288181,imprimerie Collée
22911,16796898,Imprimerie Xavier Havermans


In [40]:
dfListISSNColumns['publisher']

,autID,publisher
0,3041-5543,Weissenbruck
1,3041-5608,Concertgebouw Brugge
2,3041-5659,Prosu Media Producties
3,3041-5667,Archives générales du Royaume et Archives de l...
4,3041-5675,Archives générales du Royaume et Archives de l...
...,...,...
37404,2795-9066,Le Taureau
37405,0779-3235,Le journal des enfants
37406,3041-7929,Cinéac
37407,3041-7619,Averbode


In [41]:
dfPublisher = pd.concat([
    dfListKBRColumns['publisher'][['autID', 'publisher']],
    dfListISSNColumns['publisher'][['autID', 'publisher']]
], ignore_index=True)

In [42]:
dfPublisher = utils.normalizeColumn(dfPublisher, 'publisher')

In [43]:
dfPublisher

,autID,publisher
0,15145431,universiteit van leuven. aardrijkskundig insti...
1,18584980,koninklijke bibliotheek van belgie
2,18585150,koninklijke bibliotheek van belgie
3,16474587,koninklijke bibliotheek van belgie
4,16474568,bibliotheque royale de belgique
...,...,...
60317,2795-9066,le taureau
60318,0779-3235,le journal des enfants
60319,3041-7929,cineac
60320,3041-7619,averbode


In [44]:
dfMatching = dfMatching.merge(dfPublisher, how='left', left_on='autID', right_on='autID')

### 2.4 Classification

We will have one or more genre classifications. To be able to match based on each mentioned genre, we also have to duplicate a row, meaning that we even get more variants. For example, if we would have 3 UDC classifications, then we would generate rows with combinations of each title variant with each genre, thus `4*3=12` rows. And each of the 12 rows is compared to all ISSN+ titles! This does not scale very well, but our input data is relatively small, additionally we configure Splink to avoid unnecessary comparisons, e.g. those 4 (or 12) variants with each other, because we are not interested in duplicate detection. 


In [45]:
# todo

In [46]:
# Break down classifications such as '654.165 (05) (493.2 B.)'

### 2.5 URL
Nothing to standardize for URLs, we simply take them as they are



In [47]:
dfURL = pd.concat([
    dfListKBRColumns['url'][['autID', 'url']],
    dfListISSNColumns['url'][['autID', 'url']]
], ignore_index=True)

In [48]:
dfMatching = dfMatching.merge(dfURL, how='left', left_on='autID', right_on='autID')

In [49]:
dfMatching.notna().sum()

autID         254533
dataSource    254533
title         254520
place         249157
publisher     243708
url             1240
dtype: int64

## 3 Matching

In [50]:
dfMatching = dfMatching.rename(columns={'autID': 'unique_id'})

In [51]:
dfMatching

,unique_id,dataSource,title,place,publisher,url
0,15145431,KBR,bulletin de la societe belge d'etudes geograph...,leuven,universiteit van leuven. aardrijkskundig insti...,https://www.belgicaperiodicals.be/link/opac/15...
1,15145431,KBR,bulletin de la societe belge d'etudes geograph...,leuven,universiteit van leuven. aardrijkskundig insti...,https://www.belgicaperiodicals.be/link/opac/15...
2,15145431,KBR,tijdschrift van de belgische vereniging voor a...,leuven,universiteit van leuven. aardrijkskundig insti...,https://www.belgicaperiodicals.be/link/opac/15...
3,15145431,KBR,tijdschrift van de belgische vereniging voor a...,leuven,universiteit van leuven. aardrijkskundig insti...,https://www.belgicaperiodicals.be/link/opac/15...
4,15145431,KBR,bevas,leuven,universiteit van leuven. aardrijkskundig insti...,https://www.belgicaperiodicals.be/link/opac/15...
...,...,...,...,...,...,...
254528,3041-7619,ISSN,lenteweelde. j.v.k.a. orgaan voor katholieke s...,averbode,averbode,NaN
254529,3041-7619,ISSN,lenteweelde. j.v.k.a. orgaan voor katholieke s...,averbode,averbode,NaN
254530,3041-8038,ISSN,jeugdland.,borgerhout,jos. wils,NaN
254531,3041-8038,ISSN,voor klein en groot,borgerhout,jos. wils,NaN


### 3.1 Exploratory analysis

In [52]:
completeness_chart(dfMatching, db_api=db_api)

alt.LayerChart(...)

In [53]:
from splink.exploratory import profile_columns

profile_columns(dfMatching, db_api=DuckDBAPI(), top_n=10, bottom_n=5)

alt.VConcatChart(...)

### 3.2 Blocking

Splink basically creates a large number of possible comparisons. Since the number of comparisons scales quadratically, it is important to limit the number of comparisons as much as possible. Therefore, one can define so-called _blocking rules_ which define which columns should not be compared (e.g. because it would not be a meaningful comparison anyway).

Splink offers several ways to analyze the number of possible comparisons for specific rules. Like this it is possible to find the most ideal (combination of) rules. It is important to avoid too strict rules (to avoid missing out real matches) and, as said, limit the number of overal comparisons to avoid that the computation takes ages.

A total of **3 to 10 rules** is recommended by the Splink tutorial.

In [54]:
from splink import DuckDBAPI, block_on, splink_datasets
from splink.blocking_analysis import count_comparisons_from_blocking_rule
from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
)
from splink.blocking_analysis import n_largest_blocks

The following function checks the top 5 "block count" (number of record pairs generated by a rule) of a rule which takes combinations of publisher and place. We see millions of pairs for each combination which, as a rule on itself, is not good. This makes sense, if two records share the same publisher and place, we cannot make a distinction between two records based on the publisher/place combination. In other words: **this field pair is not useful to separate entities**.

In [55]:
result = n_largest_blocks(    table_or_tables=dfMatching,
    blocking_rule= block_on("publisher", "place"),
    link_type="dedupe_only",
    db_api=db_api,
    n_largest=15
    )

result.as_pandas_dataframe()

,key_0,key_1,count_l,count_r,block_count
0,brepols,turnhout,2329,2329,5424241
1,vanden broele,bruges,1106,1106,1223236
2,ced. samsom,diegem,1106,1106,1223236
3,ced-samsom,diegem,826,826,682276
4,larcier,brussels,773,773,597529
5,politeia,brussels,738,738,544644
6,bruylant,brussels,726,726,527076
7,dupuis,marcinelle,692,692,478864
8,stad antwerpen,antwerp,688,688,473344
9,peeters,leuven,666,666,443556


In this example we also see by chance that the data can still be normalized better. It seems that `ced. samsom` and `ced-samsom` are the same publisher (both publish in diegem, this is likely a spelling mistake). Let's fix it!

In [56]:
dfMatching.loc[dfMatching['publisher'] == 'ced-samsom', 'publisher'] = 'ced. samsom'

In [57]:
result = n_largest_blocks(    table_or_tables=dfMatching,
    blocking_rule= block_on("publisher", "place"),
    link_type="dedupe_only",
    db_api=db_api,
    n_largest=15
    )

result.as_pandas_dataframe()

,key_0,key_1,count_l,count_r,block_count
0,brepols,turnhout,2329,2329,5424241
1,ced. samsom,diegem,1932,1932,3732624
2,vanden broele,bruges,1106,1106,1223236
3,larcier,brussels,773,773,597529
4,politeia,brussels,738,738,544644
5,bruylant,brussels,726,726,527076
6,dupuis,marcinelle,692,692,478864
7,stad antwerpen,antwerp,688,688,473344
8,peeters,leuven,666,666,443556
9,kluwer,mechelen,581,581,337561


The following shows the results for a possible blocking rule on the URL column. A URL in our dataset does not uniquely identify a periodical, but more like a publisher homepage. Hence it is not very unique. If it would be unique, a single blocking rule would allow confident matches. Based on the number of comparisons it would not harm to have such a rule: as we see, 1024 comparions at max, which is nothing compared to all 888 million comparisons.

In [58]:
result = n_largest_blocks(    table_or_tables=dfMatching,
    blocking_rule= block_on("url"),
    link_type="dedupe_only",
    db_api=db_api,
    n_largest=15
    )

result.as_pandas_dataframe()

,key_0,count_l,count_r,block_count
0,http://cdserver/NetManBin/nmwebclt.asp?CONFIGI...,32,32,1024
1,www.eurodb.be,30,30,900
2,http://firstsearch.oclc.org,18,18,324
3,http://www.om-mp.be/stat/corr/start/f/home.html,18,18,324
4,http://cdserver/NetManBin/nmwebclt.asp?CONFIGI...,16,16,256
5,https://www.belgicaperiodicals.be/link/opac/15...,15,15,225
6,www.filmarchief.be,12,12,144
7,www.ordredespharmaciens.be,12,12,144
8,http://www.deboecksuperieur.com/collection/oxalis,12,12,144
9,www.cinematheque.be,12,12,144


Let's compare combinations of blocking rules.

In [59]:
blocking_rules_for_analysis = [
    "l.dataSource != r.dataSource AND l.title = r.title",
    "l.dataSource != r.dataSource AND l.url = r.url"
]


cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=dfMatching,
    blocking_rules=blocking_rules_for_analysis,
    db_api=db_api,
    link_type="dedupe_only",
)

alt.Chart(...)

### 3.4 Estimating model parameters

The goal is to learn the relative importance of different data fields to find a match. As the Splink tutorial states

> A match on date of birth is a much stronger indicator that two records refer to the same entity than a match on gender.

Thus our goal is to learn good match weights with the probabilistic model. What we have to do is defining when something is "similar". In Splink this is done by using _Comparisons_  (given a column name) and _Comparison Levels_ (exacvt match, x characters difference, all other). 

In the following we specify the linkage model.

In [60]:
from splink import Linker, SettingsCreator, block_on, DuckDBAPI
import splink.comparison_library as cl

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.LevenshteinAtThresholds("publisher",2),
        cl.LevenshteinAtThresholds("title",2),
        cl.ExactMatch("place").configure(term_frequency_adjustments=True)
    ],
    blocking_rules_to_generate_predictions=blocking_rules_for_analysis,
    retain_intermediate_calculation_columns=True,
)

linker = Linker(dfMatching, settings, db_api=DuckDBAPI())

Now we have to specify the two parameters `probability_two_random_records_match` and `u`. We can specify the first by computing the high precision matches (look which records hafe the same title), a rough estimate and the total possible comparisons.

In [61]:
probability_two_random_records_match = (0.9* 24000)/(24000*37000)
probability_two_random_records_match

2.4324324324324324e-05

In [62]:
# takes around 2 minutes
linker.training.estimate_u_using_random_sampling(max_pairs=1e8)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - publisher (no m values are trained).
    - title (no m values are trained).
    - place (no m values are trained).


In [63]:
# just in a loop takes to long, let's do it step by step and save in between
blocking_rules_for_m_estimation = [
    "l.dataSource != r.dataSource AND l.title = r.title",
    "l.dataSource != r.dataSource AND l.publisher = r.publisher",
    "l.dataSource != r.dataSource AND l.place = r.place"
]

In [64]:
# first iteration with title blocking rule takes around 3 seconds
training_session = (
    linker.training.estimate_parameters_using_expectation_maximisation(blocking_rules_for_m_estimation[0])
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.dataSource != r.dataSource AND l.title = r.title

Parameter estimates will be made for the following comparison(s):
    - publisher
    - place

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - title

Iteration 1: Largest change in params was -0.505 in the m_probability of publisher, level `Exact match on publisher`
Iteration 2: Largest change in params was 0.118 in the m_probability of publisher, level `All other comparisons`
Iteration 3: Largest change in params was 0.0609 in the m_probability of publisher, level `All other comparisons`
Iteration 4: Largest change in params was 0.0378 in the m_probability of publisher, level `All other comparisons`
Iteration 5: Largest change in params was 0.0241 in the m_probability of publisher, level `All other comparisons`
Iteration 6: Largest change in params was 0.0157 in th

In [65]:
# second iteration takes around 4 minutes
training_session = (
    linker.training.estimate_parameters_using_expectation_maximisation(blocking_rules_for_m_estimation[1])
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.dataSource != r.dataSource AND l.publisher = r.publisher

Parameter estimates will be made for the following comparison(s):
    - title
    - place

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - publisher

Iteration 1: Largest change in params was -0.892 in the m_probability of title, level `Exact match on title`
Iteration 2: Largest change in params was 0.134 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.159 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.0511 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.0118 in probability_two_random_records_match
Iteration 6: Largest change in params was 0.00264 in probability_two_random_records_match
Iteration 7: Largest change in params was 0.000718 in probability

The two iterations above where enough, we should see the message

> Your model is fully trained. All comparisons have at least one estimate for their m and u values

In the following we will look into some visualisations. When hovering over the graphs you can zoom in and out. These are general visualisations on the model weights and the training process. In the next section we apply the model on the data and visualize some concrete predictions.

In [66]:
linker.visualisations.match_weights_chart()

/home/slieber/repos/kbr/issn/issn-matching/py-issn-matching/lib/python3.12/site-packages/altair/vegalite/v6/api.py:4124: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  return _tp.from_dict(dct, validate=validate)


alt.VConcatChart(...)

The prior weight is negative and read, meaning that in a Bayesian sense represents our belief that two records taken at random have a match. Based on the prior, we look into the other columns and change our belief on based what we see. 
If there is an exact match on publisher name we change our belief that two records might be a match. We do the same when there is a partial match with a short Levenshtein distance, however, the belief changes less than with an exact match.

Apparently the model puts more weight on a similar publisher than on a matching title.

In [67]:
linker.visualisations.m_u_parameters_chart()

alt.HConcatChart(...)

In [68]:
linker.visualisations.parameter_estimate_comparisons_chart()

alt.Chart(...)

In [70]:
settings = linker.misc.save_model_to_json(trainedModelFilename, overwrite=True)

## 4 Predicting results

In [73]:
dfPredictions = linker.inference.predict(threshold_match_probability=0.2)

Blocking time: 0.06 seconds
Predict time: 0.04 seconds


In [74]:
dfPredictions.as_pandas_dataframe(limit=10)

,match_weight,match_probability,unique_id_l,unique_id_r,publisher_l,publisher_r,gamma_publisher,bf_publisher,title_l,title_r,...,gamma_place,tf_place_l,tf_place_r,bf_place,bf_tf_adj_place,url_l,url_r,dataSource_l,dataSource_r,match_key
0,6.168341,0.986287,1375-1840,18540733,ced. samsom,ced. samsom,2,329.930576,loonadministratie,loonadministratie,...,1,0.013028,0.013028,3.428973,19.962104,None,None,ISSN,KBR,0
1,6.168341,0.986287,1375-1840,18540733,ced. samsom,ced. samsom,2,329.930576,loonadministratie,loonadministratie,...,1,0.013028,0.013028,3.428973,19.962104,None,None,ISSN,KBR,0
2,6.168341,0.986287,1375-1654,18540888,ced. samsom,ced. samsom,2,329.930576,administration des salaires,administration des salaires,...,1,0.013028,0.013028,3.428973,19.962104,None,None,ISSN,KBR,0
3,6.168341,0.986287,1375-1654,18540888,ced. samsom,ced. samsom,2,329.930576,administration des salaires,administration des salaires,...,1,0.013028,0.013028,3.428973,19.962104,None,None,ISSN,KBR,0
4,6.168341,0.986287,1372-6307,18542084,ced. samsom,ced. samsom,2,329.930576,modeles et contrats de societes,modeles et contrats de societes,...,1,0.013028,0.013028,3.428973,19.962104,None,None,ISSN,KBR,0
5,6.168341,0.986287,1372-6307,18542084,ced. samsom,ced. samsom,2,329.930576,modeles et contrats de societes,modeles et contrats de societes,...,1,0.013028,0.013028,3.428973,19.962104,None,None,ISSN,KBR,0
6,5.216453,0.973808,1372-6315,18542168,ced.-samsom,ced. samsom,1,170.559432,lexique societes commerciales,lexique societes commerciales,...,1,0.013028,0.013028,3.428973,19.962104,None,None,ISSN,KBR,0
7,5.216453,0.973808,1372-6315,18542168,ced.-samsom,ced. samsom,1,170.559432,lexique societes commerciales,lexique societes commerciales,...,1,0.013028,0.013028,3.428973,19.962104,None,None,ISSN,KBR,0
8,6.809727,0.991165,1781-2038,18542903,kluwer,kluwer,2,329.930576,zakboekje overheidsopdrachten,zakboekje overheidsopdrachten,...,1,0.008352,0.008352,3.428973,31.137428,None,None,ISSN,KBR,0
9,1.808933,0.777967,1781-2038,18542903,kluwer,kluwer,2,329.930576,zakboekje overheidsopdrachten,wetboek,...,1,0.008352,0.008352,3.428973,31.137428,None,None,ISSN,KBR,0


In [75]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    dfPredictions, threshold_match_probability=0.5
)
clusters.as_pandas_dataframe(limit=10)

Completed iteration 1, num edges remaining to process: 23940
Completed iteration 2, num edges remaining to process: 0


,cluster_id,unique_id,dataSource,title,place,publisher,url
0,2034-5542,2034-5542,ISSN,jungle kids,brussels,jungle,None
1,2034-5550,2034-5550,ISSN,l' ecole au quotidien.,charleroi,couleur livres,None
2,2034-5550,2034-5550,ISSN,l' ecole au quotidien.,charleroi,couleur livres,None
3,2034-5550,2034-5550,ISSN,collection l'ecole au quotidien,charleroi,couleur livres,None
4,2034-5550,2034-5550,ISSN,collection l'ecole au quotidien,charleroi,couleur livres,None
5,2034-5550,2034-5550,ISSN,l' ecole au quotidien,charleroi,couleur livres,None
6,2034-5550,2034-5550,ISSN,l' ecole au quotidien,charleroi,couleur livres,None
7,2034-5569,2034-5569,ISSN,collection signe.,brussels,le lombard,None
8,2034-5569,2034-5569,ISSN,collection signe.,brussels,le lombard,None
9,2034-5569,2034-5569,ISSN,"collection ""signe""",brussels,le lombard,None


In [77]:
records_to_view = dfPredictions.as_record_dict(limit=30)
linker.visualisations.waterfall_chart(records_to_view, filter_nulls=False)

alt.LayerChart(...)

In [82]:
from IPython.display import IFrame
linker.visualisations.comparison_viewer_dashboard(dfPredictions, "comparison.html", overwrite=True)

In [83]:
IFrame(src="./comparison.html", width="100%", height=1200)